In [1]:
import json
import nltk
import re
import ipywidgets as widgets
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

nltk.download(['punkt', 'stopwords'], quiet=True);

In [2]:
def tokenize(doc):
    tokens = []
    sentences = nltk.sent_tokenize(doc)
    for sentence in sentences:
        tokens.append(nltk.word_tokenize(sentence))
    
    return tokens

In [3]:
filename = 'data/processed_reviews.json'
with open(filename, 'rb') as file:
    reviews = json.load(file)

filename = 'data/processed_games.json'
with open(filename, 'rb') as file:
    games = json.load(file)

In [ ]:
corpus = set()
stemmer = nltk.stem.snowball.EnglishStemmer()
stopwords = nltk.corpus.stopwords.words('english')
products = [g['id'] for g in games]
for product in products:
    for review in reviews:
        if review['product_id'] == product:
            for sentence in tokenize(review['text']):
                for word in sentence:
                    word = word.lower()
                    if word not in stopwords and word.isalnum():
                        word = stemmer.stem(word)
                        corpus.add(word)

In [ ]:
X = []
stemmer = nltk.stem.snowball.EnglishStemmer()
stopwords = nltk.corpus.stopwords.words('english')
products = [g['id'] for g in games]
for product in products:
    bag_of_words = set()
    for review in reviews:
        if review['product_id'] == product:
            for sentence in tokenize(review['text']):
                for word in sentence:
                    word = word.lower()
                    if word not in stopwords and word.isalnum():
                        word = stemmer.stem(word)
                        bag_of_words.add(word)
                        
    X.append([int(word in bag_of_words) for word in corpus])

In [ ]:
Y = cosine_similarity(X)

# Creating a mask so argmax does not cause product to return itself.
n = len(Y)
Y_mask = [list([False] * n) for y in Y]
for i in range(n):
    Y_mask[i][i] = True

Y = np.ma.MaskedArray(Y, Y_mask)

In [ ]:
def f(i):
    
    return  [np.max(Y[i]), games[np.argmax(Y[i])]]

dropdown = widgets.Dropdown(
    options=[(g['app_name'], games.index(g)) for g in games],
    description='Index:',
)
widgets.interact(f, i=dropdown);